## install required packages
run the following command line in terminal

       pip install -r requirements.txt

## Import packages

In [1]:
import pandas as pd
import numpy as np
import os
import json
import obonet
import inflect
import networkx as nx
import matplotlib.pyplot as plt
import requests
import re
from openai import OpenAI
from pydantic import BaseModel
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Define output format
cell type ( string )

In [2]:
class cellTypeFormat(BaseModel):
       cellType: str

## Read cell clusters file 
file path set to ./Data/all.csv

In [99]:
def read_cluster(path):
      dataframe = pd.read_csv(path)
      print(dataframe['marker'])
      return dataframe 
GENE_LIST = read_cluster('./Data/all.csv')
print(gene_list.iloc[0])

0       MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
1       MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...
2       IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...
3       IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...
4       GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...
                              ...                        
1125    KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...
1126    JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...
1127    TFF3,PKHD1L1,PROX1,NTS,FLT4,RELN,GPR182,STAB2,...
1128    VWF,ACKR1,PLVAP,PECAM1,AQP1,RAMP3,CLEC14A,SOX1...
1129    MSLN,UPK3B,MUC16,CALB2,KLK11,BICDL1,CHAC1,TGM1...
Name: marker, Length: 1130, dtype: object
dataset                                                               Azimuth
tissue                                                                   PBMC
marker                      MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
manual_annotation                                         Intermediate B cell
manual_C

## Split genelist
split genelist to multiple chunks so that it won't exceed model input size

In [101]:
# Generate the user message for each chunk
def generate_user_message(gene_list:  pd.DataFrame, max_chunk_size=100) -> list:
    # Split the gene_list into chunks
    chunks = [group for _, group in gene_list.groupby(gene_list.index // max_chunk_size)]
    messages = []
    for chunk in chunks:
        # Construct the user message for each chunk
        user_message = (
            chunk['tissue'] + ' : '
            + chunk['marker']
        )
        messages.append(user_message)
    return messages

# Generate messages with a tunable chunk size
messages = generate_user_message(GENE_LIST, max_chunk_size=10)  # Adjust max_chunk_size as needed
print(messages[0])

0    PBMC : MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD7...
1    PBMC : MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, ...
2    PBMC : IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, ...
3    PBMC : IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, T...
4    PBMC : GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, G...
5    PBMC : TCF7, CD4, CCR7, IL7R, FHIT, LEF1, MAL,...
6    PBMC : MKI67, TOP2A, PCLAF, CENPF, TYMS, NUSAP...
7    PBMC : IL7R, TMSB10, CD4, ITGB1, LTB, TRAC, AQ...
8    PBMC : IL7R, CCL5, FYB1, GZMK, IL32, GZMA, KLR...
9    PBMC : RTKN2, FOXP3, AC133644.2, CD4, IL2RA, T...
dtype: object


## Gemeni ai

### model configuration
set the GOOGLE_API_KEY under .env file

In [102]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
gemeni_model = genai.GenerativeModel("gemini-1.5-flash")
gemeni_config = genai.GenerationConfig(response_mime_type="application/json",response_schema=list[cellTypeFormat])

### Send prompt to gemini

In [103]:
earlyStop = 2
Prompt = "Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types."
def annotateCell_Gemini(messages: list)->list:
    global gemeni_model, gemeni_config 
    responses = []
    i = 0
    for message in messages:
       i+=1
       if i> earlyStop:
              break
       prompt = Prompt
       for cluster in message:
              prompt += cluster
              prompt += '\n'
       print(prompt)
       response = gemeni_model.generate_content(
              prompt,
              generation_config=gemeni_config
       )
       responses.append(response.text)
    return responses


In [104]:
annotateCell_Gemini_results = annotateCell_Gemini(messages)
print(annotateCell_Gemini_results)

Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types.PBMC : MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LINC01857, RALGPS2, BANK1, CD79B
PBMC : MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, RALGPS2, TNFRSF13C, LINC01781
PBMC : IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, TCL1A, CD79B, YBX3
PBMC : IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13B, POU2AF1, CPNE5, HRASLS2, NT5DC2
PBMC : GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2M, IL32, NKG7
PBMC : TCF7, CD4, CCR7, IL7R, FHIT, LEF1, MAL, NOSIP, LDHB, PIK3IP1
PBMC : MKI67, TOP2A, PCLAF, CENPF, TYMS, NUSAP1, ASPM, PTTG1, TPX2, RRM2
PBMC : IL7R, TMSB10, CD4, ITGB1, LTB, TRAC, AQP3, LDHB, IL32, MAL
PBMC : IL7R, CCL5, FYB1, GZMK, IL32, GZMA, KLRB1, TRAC, LTB, AQP3
PBMC : RTKN2, FOXP3, AC133644.2, CD4, IL2RA, TIGIT, CTLA4, FCRL3, LAIR2, IKZF2

Identify cell types using the following tissue name 

## GPT

### model configuration

In [8]:
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

### upload file

In [ ]:
def upload_cell_cluster(filename):
       global client
       client.files.create(
              file=open(filename, "rb"),
              purpose="assistant"
       )
       print(client.files.list())

### send prompt to GPT

In [7]:
def annotateCell_GPT(messages: list)->list:
    global client
    responses = []
    for message in messages:
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content":  "Identify cell types using the following tissue name and markers separately for each row. "
            + "Only provide the cell type name. "
            + "Do not show numbers before the name. "
            + "Some can be a mixture of multiple cell types."},
                {"role": "user", "content": message}
            ],
            response_format=cellTypeFormat,
        )
        responses.append(completion.choices[0].message.parsed)
    return responses


## Scoring system

### result parsing

In [107]:
def json_parsing(annotateCell_results: json)->list:
       # Parse the strings into Python objects
       parsed_results = [json.loads(item.strip()) for item in annotateCell_results]
       # Flatten the results to get a list of all cell types
       cell_types = [entry['cellType'] for result in parsed_results for entry in result]
       return cell_types

In [108]:
cell_types = json_parsing(annotateCell_Gemini_results)
print(cell_types)

['B cells', 'B cells', 'B cells', 'B cells', 'Cytotoxic T cells', 'T cells', 'Proliferating cells', 'T cells', 'T cells', 'T cells', 'T cells, Neutrophils', 'T cells', 'T cells', 'T cells', 'Monocytes', 'Monocytes', 'Dendritic cells', 'B cells', 'Monocytes', 'Monocytes']


### CL correspondence
match the cell_types to get CLID from Cell Ontology
search name in graph and return the corresponding clid\
if no matches found, send request to cell ontology

In [105]:
# URL for Cell Ontology (CO) OBO file
CO_URL = 'http://purl.obolibrary.org/obo/CL.obo'
OBO_FILE_PATH = 'oboNet/cl.obo'
# dictionary to store search result
NAME_TO_CLID_DICT = dict()

def load_ontology(url):
    graph = obonet.read_obo(url)
    return graph
def search_ontology(cell_name: str, ontology='cl')->str:
    global NAME_TO_CLID_DICT
    normalized_name = inflector.singular_noun(cell_name.lower()) or cell_name.lower()
    normalized_name = re.sub(r"^\(?\d+\)?\.", "", normalized_name).strip()
    # search in dict for faster access to clid and name
    if normalized_name in NAME_TO_CLID_DICT :
        return NAME_TO_CLID_DICT[normalized_name]
    
    for id, data in GRAPH.nodes(data=True):
        if 'name' in data:
            # Singularize and normalize the GRAPH's 'name'
            label = inflector.singular_noun(data['name'].lower()) or data['name'].lower()
            # Compare normalized names
            if label == normalized_name:
                NAME_TO_CLID_DICT[normalized_name] = (id, label)
                return id, label # Return the clid and label(name) if a match is found
    
    # If no clid name found in GRAPH, send request to ontology api    
    # OLS API URL
    url = f"https://www.ebi.ac.uk/ols/api/search?q={normalized_name}&ontology={ontology}"

    # Make the API request
    response = requests.get(url)
    if response.status_code == 200:
        if len(response.json()['response']['docs'])==0:
          return None, None
        id = response.json()['response']['docs'][0]['obo_id']
        label = response.json()['response']['docs'][0]['label']
        NAME_TO_CLID_DICT[normalized_name] = (id, label)
        return id, label
    else:
        print(f"Error: {response.status_code}")
        return None, None

inflector = inflect.engine()
# Load the Cell Ontology
GRAPH = load_ontology(OBO_FILE_PATH)



In [109]:
clids = []
for cell_type in cell_types:
  clids.append(search_ontology(cell_type))
print(clids)

[('CL:0000236', 'b cell'), ('CL:0000236', 'b cell'), ('CL:0000236', 'b cell'), ('CL:0000236', 'b cell'), ('CL:0000910', 'cytotoxic t cell'), ('CL:0000084', 't cell'), ('CL:4033068', 'cycling B cell'), ('CL:0000084', 't cell'), ('CL:0000084', 't cell'), ('CL:0000084', 't cell'), ('CL:0000775', 'neutrophil'), ('CL:0000084', 't cell'), ('CL:0000084', 't cell'), ('CL:0000084', 't cell'), ('CL:0000576', 'monocyte'), ('CL:0000576', 'monocyte'), ('CL:0000451', 'dendritic cell'), ('CL:0000236', 'b cell'), ('CL:0000576', 'monocyte'), ('CL:0000576', 'monocyte')]


### Calculate distance with two CLID
#### Method 1 : shortest path length between two nodes
calculate the distance between two nodes ( manual CLID and LLM annotated CLID)

##### Function Definition
- input
    - GRAPH : graph with relationship between cells (i.e. GRAPH)
    - clid or name : two nodes to assign score
- output
    - int : distance of the shortest path length of two nodes, -1 for no path

In [110]:
def calculate_difference(graph:nx.graph, clid_1:str, clid_2:str)->int:
    try:
        return nx.shortest_path_length(graph, source=clid_1, target=clid_2)
    except:
        return -1
def calculate_difference_name(graph:nx.graph, type_1:str, type_2:str)->int:
    clid_1, label_1 = search_ontology(type_1)
    clid_2, label_2 = search_ontology(type_2)
    return calculate_difference(graph, clid_1, clid_2)


In [111]:
print(calculate_difference(GRAPH, 'CL:0002250', 'CL:0009016'))
print(calculate_difference(GRAPH,  'CL:0009016','CL:0002250'))
# intestinal crypt stem cell 0002250
# intestinal crypt stem cell of large intestine 0009016
# should put the broader type (i.e. LLM annotated ) behind
print(calculate_difference_name(GRAPH, 'T cells', 'Cytotoxic T cells'))
print(calculate_difference_name(GRAPH, 'Cytotoxic T cells', 'T cells'))
print(calculate_difference_name(GRAPH, 'T-helper cells', 'T cells'))

-1
1
-1
3
3


#### Method 2 : calculate adjusted path-based similarity
assign score according to 'On measuring of SImilarity between tree nodes'

$$ l_a (v_i, v_j) = \frac{l(v_i, lca_{ij})+l(v_j, lca_{ij})}{1 + l(lca_{ij},t)}$$

while score equals $$ s_a(v_i, v_j) = \frac{1}{1+l_a(v_i, v_j)} = \frac{1 + l(lca_{ij},t)}{1+ l(v_i, lca_{ij})+l(v_j, lca_{ij}) + l(lca_{ij},t)} $$

##### Trim graph
only keep nodes starting with CL, and keep edges 'is_a'

In [112]:
TRIMMED_GRAPH = nx.DiGraph()
i = 0
for clid in GRAPH.nodes:
       entry = GRAPH.nodes[clid]
       if 'is_a' in entry:
              for parent in entry['is_a']:
                     if parent[0:2] == 'CL' and clid[0:2]=='CL':
                            TRIMMED_GRAPH.add_node(clid) 
                            TRIMMED_GRAPH.add_edge(parent, clid)  # Add directed edge
print(nx.is_directed_acyclic_graph(TRIMMED_GRAPH))
TRIMMED_GRAPH.remove_edge('CL:0000164','CL:0000163')
print(nx.is_directed_acyclic_graph(TRIMMED_GRAPH))


False
True


##### Function definition
- input
    - graph : DAG graph (i.e. G_scc)
    - scc_mapping : dictionary to map clid to scc (i.e. SCC_MAPPING)
    - clid or name : two nodes to assign score
- output
    - float : adjusted score taking depth into consideration


In [118]:
def adjusted_path_based_score(graph:nx.DiGraph,  clid_1:str, clid_2:str)->float:
        try:
                ugraph = graph.to_undirected()
                lca = nx.lowest_common_ancestor(graph, clid_1, clid_2)
                root_nodes = [x for x in graph.nodes() if graph.out_degree(x)>1 and graph.in_degree(x)==0]
                l_i_lca = nx.shortest_path_length(ugraph, source = clid_1, target = lca)
                l_j_lca = nx.shortest_path_length(ugraph, source = clid_2, target = lca)
                l_lca_t = 0
                for root_node in root_nodes:
                        if(nx.has_path(ugraph, source=lca, target=root_node)):
                             l_lca_t = nx.shortest_path_length(ugraph, source = lca, target = root_node)
                return format(((1+l_lca_t) / (1+l_i_lca + l_j_lca + l_lca_t)), '.4f')   # give 2 digits after the point
        except:
                return -1

    
def adjusted_path_based_score_name(graph:nx.DiGraph,  type_1:str, type_2:str)->float:
        ugraph = graph.to_undirected()
        # convert name to clid
        clid_1, label_1 = search_ontology(type_1)
        clid_2, label_2 = search_ontology(type_2)
        return adjusted_path_based_score(graph, clid_1=clid_1, clid_2=clid_2)



In [119]:
print(adjusted_path_based_score(TRIMMED_GRAPH, 'CL:0000910', 'CL:0000898'))
print(adjusted_path_based_score_name(TRIMMED_GRAPH, 'Cytotoxic T cells', 'T cells'))


0.7000
0.6667


In [131]:
diff_result=[]
for i in range(len(clids)):
  clid_1, label_1 =clids[i] #LLM annotated CLID
  label_2 =GENE_LIST["manual_annotation"][i] 
  difference = calculate_difference_name(GRAPH, label_2, label_1)
  diff_result.append(difference)
  print(label_1, label_2)
  # print(difference)
print(diff_result)

b cell Intermediate B cell
b cell Memory B cell
b cell Naive B cell
b cell Plasmablast
cytotoxic t cell CD4+ Cytotoxic T
t cell CD4+ Naive T
cycling B cell CD4+ Proliferating T
t cell CD4+ Central Memory T
t cell CD4+ Effector Memory T
t cell Regulatory T
neutrophil CD8+ Naive T
t cell CD8+ Proliferating T
t cell CD8+ Central Memory T
t cell CD8+ Effector Memory T
monocyte AXL+ Dendritic Cell
monocyte Conventional Dendritic Cell 1
dendritic cell Conventional Dendritic Cell 2
b cell Plasmacytoid Dendritic Cell
monocyte CD14+ Monocyte
monocyte CD16+ Monocyte
[-1, 3, 3, 3, -1, 2, -1, 5, 5, 2, -1, 2, 5, 5, -1, -1, 1, -1, 1, 2]


In [133]:
adjusted_score_result=[]
for i in range(len(clids)):
  clid_1, label_1 =clids[i] #LLM annotated CLID
  clid_2, label_2 =GENE_LIST["manual_CLID"][i] , GENE_LIST["manual_annotation"][i]
  difference = adjusted_path_based_score_name(TRIMMED_GRAPH, label_2, label_1)
  # difference = adjusted_path_based_score(TRIMMED_GRAPH, clid_2, clid_1)

  adjusted_score_result.append(difference)
  print(label_1, label_2, sep='\t')
print(adjusted_score_result)

b cell	Intermediate B cell
b cell	Memory B cell
b cell	Naive B cell
b cell	Plasmablast
cytotoxic t cell	CD4+ Cytotoxic T
t cell	CD4+ Naive T
cycling B cell	CD4+ Proliferating T
t cell	CD4+ Central Memory T
t cell	CD4+ Effector Memory T
t cell	Regulatory T
neutrophil	CD8+ Naive T
t cell	CD8+ Proliferating T
t cell	CD8+ Central Memory T
t cell	CD8+ Effector Memory T
monocyte	AXL+ Dendritic Cell
monocyte	Conventional Dendritic Cell 1
dendritic cell	Conventional Dendritic Cell 2
b cell	Plasmacytoid Dendritic Cell
monocyte	CD14+ Monocyte
monocyte	CD16+ Monocyte
['0.2000', '0.7000', '0.7000', '0.7000', '0.5833', '0.7500', '0.4545', '0.5455', '0.5455', '0.7500', '0.2727', '0.7500', '0.5455', '0.5455', '0.5000', '0.5714', '0.8333', '0.4000', '0.8000', '0.6667']


### Assign new scoring system to all.csv

In [39]:
MODELS = ['gpt4aug3', 'gpt4mar23','gpt3.5aug3', 'CellMarker2.0','SingleR', 'ScType']
MANUAL = 'manual'

In [ ]:
def assign_score(gene_list: pd.DataFrame):
       global MODELS
       global MANUAL
       global GRAPH
       gene_list_scored = pd.DataFrame()
       gene_list_scored = gene_list.copy()
       for MODEL in MODELS:
              scores = []
              for i in range(len(gene_list)):
                     clid_1 = gene_list[MANUAL+'_CLID'][i] 
                     clid_2 = gene_list[MODEL+'_CLID'][i]
                     scores.append(calculate_difference(GRAPH, clid_1, clid_2))
              gene_list_scored[MODEL+'_aggrement_modified'] = scores
       return gene_list_scored


In [ ]:
gene_list_scored = assign_score(GENE_LIST)
print(gene_list_scored)

            dataset       tissue  \
0           Azimuth         PBMC   
1           Azimuth         PBMC   
2           Azimuth         PBMC   
3           Azimuth         PBMC   
4           Azimuth         PBMC   
...             ...          ...   
1125  tabulasapiens  Vasculature   
1126  tabulasapiens  Vasculature   
1127  tabulasapiens  Vasculature   
1128  tabulasapiens  Vasculature   
1129  tabulasapiens  Vasculature   

                                                 marker  \
0     MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...   
1     MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...   
2     IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...   
3     IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...   
4     GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...   
...                                                 ...   
1125  KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...   
1126  JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...   
1127  TFF3,PKHD1L1,PROX1,NTS,FLT4,RE

In [42]:
gene_list_scored.to_csv('Data/all_modified.csv')

## Supplement

#### Build new graph with SCC
create scc and scc mapping to convert graph to DAG

In [ ]:
trimmed_graph_copy= TRIMMED_GRAPH.copy()
# Find the SCCs using networkx's strongly_connected_components
sccs = list(nx.strongly_connected_components(trimmed_graph_copy))
G_SCC = nx.DiGraph()

# create scc mapping to map clid to group
SCC_MAPPING = {}
for i, scc in enumerate(sccs):
    for node in scc:
        SCC_MAPPING[node] = i

for u, v in trimmed_graph_copy.edges():
    if SCC_MAPPING[u] != SCC_MAPPING[v]:  # Only add edges between different SCCs
        G_SCC.add_edge(SCC_MAPPING[v], SCC_MAPPING[u])
# Check if the new graph is a DAG (should return True)
print("Is DAG?", nx.is_directed_acyclic_graph(G_SCC))

#### Build reverse mapping
note that only two nodes are in the same group(CL:0000164, CL:0000163)
- CL:0000164 eternoendocrine cell
- CL:0000163 endocrine cell

Endocrine cell is_a eteroendocrine cell seems to be a incorrect edge

In [ ]:
reverse_mapping = {}
for items in SCC_MAPPING:
       # print(items, SCC_MAPPING[items])
       if(SCC_MAPPING[items] in reverse_mapping):
              reverse_mapping[SCC_MAPPING[items]].append(items)
              print(reverse_mapping[SCC_MAPPING[items]])
       else:
              reverse_mapping[SCC_MAPPING[items]] = [items]

In [ ]:
print(GRAPH.nodes['CL:0000164'])
print(GRAPH.nodes['CL:0000163'])